In [331]:
import os
import pandas as pd
import numpy as np
def answer_one():
    energy = pd.read_excel('Energy Indicators.xls', skiprows=17, skipfooter=38)
    energy = energy.iloc[:, 2:]
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy['Energy Supply'] = energy['Energy Supply'] * 1_000_000
    energy.replace('...', np.NaN, inplace=True)

    energy['Country'] = energy['Country'].str.replace(r'\s*\([^)]*\)', '', regex=True)
    energy['Country'] = energy['Country'].str.replace(r"\d+", "", regex=True)
    energy['Country'] = energy['Country'].replace({"Republic of Korea": "South Korea",
                                                  "United States of America": "United States",
                                                   "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                                                   "China, Hong Kong Special Administrative Region": "Hong Kong"})

    GDP = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_5871885.csv', skiprows=4)
    GDP.rename(columns={'Country Name': 'Country'}, inplace=True)
    GDP = GDP.iloc[:, :-8]
    GDP['Country'] = GDP['Country'].str.replace(r'\s*\([^)]*\)', '', regex=True)
    GDP['Country'] = GDP['Country'].str.replace(r"\d+", "", regex=True)
    GDP['Country'] = GDP['Country'].replace({"Korea, Rep.": "South Korea",
                                                       "Iran, Islamic Rep.": "Iran",
                                                       "Hong Kong SAR, China": "Hong Kong"})


    ScimEn = pd.read_excel('scimagojr country rank 1996-2022.xlsx')
    ScimEn['Country'] = ScimEn['Country'].str.replace(r'\s*\([^)]*\)', '', regex=True)
    ScimEn['Country'] = ScimEn['Country'].str.replace(r"\d+", "", regex=True)

    top_15_countries = ScimEn.head(15)

    merged_data = energy.merge(GDP, on = 'Country', how = 'left')
    merged_data = top_15_countries.merge(merged_data, on = 'Country', how = 'left')

    merged_data = merged_data.set_index('Country')

    columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'] + [str(i) for i in range(2006, 2016)]
    result = merged_data[columns]

    return result
print(answer_one())


                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     360468             358777    3947871   
United States          2     199442             195042    3068926   
India                  3      76103              74167     760964   
Japan                  4      56249              55680     633294   
United Kingdom         5      52572              51156     909276   
Germany                6      47781              46767     641717   
Russian Federation     7      43567              43290     175721   
Canada                 8      39036              38276     787010   
Italy                  9      35991              34424     529459   
South Korea           10      35294              35005     503147   
Iran                  11      29776              29448     511199   
France                12      29351              28759     471469   
Spain                 13      2788

In [332]:
def answer_two():
    Top15 = answer_one()
    avgGDP = Top15.loc[:, '2006':'2015'].mean(axis=1)  
    avgGDP = avgGDP.sort_values(ascending=False)  
    return avgGDP
print(answer_two())

Country
United States         1.570403e+13
China                 6.927707e+12
Japan                 5.239642e+12
Germany               3.523342e+12
United Kingdom        2.780276e+12
France                2.691337e+12
Italy                 2.142986e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.400886e+12
South Korea           1.221372e+12
Australia             1.207513e+12
Iran                  4.563261e+11
dtype: float64


In [333]:
def answer_three():
    Top15 = answer_one()
    avgGDP = answer_two()
    largest_avgGDP_6 = avgGDP.index[5]
    GDP_2006 = Top15.loc[largest_avgGDP_6, '2006']
    GDP_2015 = Top15.loc[largest_avgGDP_6, '2015']
    GDP_change = GDP_2015 - GDP_2006
    return GDP_change
print(answer_three())


118652421857.7998


In [334]:

def answer_four():
    Top15 = answer_one()
    Top15['Citations Ratio'] = Top15['Self-citations'] / Top15['Citations']
    max_ratio = Top15['Citations Ratio'].max()
    country_max_ratio = Top15[Top15['Citations Ratio'] == max_ratio].index[0]
    return (country_max_ratio, max_ratio)
print(answer_four())


('China', 0.6853754846599598)


In [335]:
def answer_five():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15 = Top15.sort_values('Population', ascending=False)
    third_most_populous = Top15.index[2]
    return third_most_populous

print(answer_five())

United States


In [336]:
def answer_six():
    Top15 = answer_one()
    Top15['Citable documents per Capita'] = Top15['Citable documents'] / (Top15['Energy Supply'] / Top15['Energy Supply per Capita'])
    Top15['Citable documents per Capita'].replace([np.inf, -np.inf], np.nan, inplace=True)
    correlation = Top15['Citable documents per Capita'].corr(Top15['Energy Supply per Capita'], method='pearson')
    return correlation
print(answer_six())

0.7114342527331208


In [340]:

def answer_seven():
    Top15 = answer_one()
    ContinentDict = {
        'China': 'Asia',
        'United States': 'North America',
        'Japan': 'Asia',
        'United Kingdom': 'Europe',
        'Russian Federation': 'Europe',
        'Canada': 'North America',
        'Germany': 'Europe',
        'India': 'Asia',
        'France': 'Europe',
        'South Korea': 'Asia',
        'Italy': 'Europe',
        'Spain': 'Europe',
        'Iran': 'Asia',
        'Australia': 'Australia',
        'Brazil': 'South America'
    }
    Top15['Continent'] = [ContinentDict[country] for country in Top15.index]
    
    result = Top15.groupby('Continent')['Energy Supply'].agg(['size', 'sum', 'mean', 'std'])
    result.columns = ['size', 'sum', 'mean', 'std']
    
    return result

print(answer_seven())

               size           sum          mean           std
Continent                                                    
Asia              5  199549000000  3.990980e+10  4.970204e+10
Australia         1    5386000000  5.386000e+09           NaN
Europe            6   73940000000  1.232333e+10  9.482712e+09
North America     2  101269000000  5.063450e+10  5.685633e+10
South America     1   12149000000  1.214900e+10           NaN
